In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Desktop\\code\\python\\Mlops-Production-Ready-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Admin\\Desktop\\code\\python\\Mlops-Production-Ready-Deep-Learning-Project'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int 

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories


In [29]:
class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.Config = read_yaml(config_file_path)
        self.Params = read_yaml(params_file_path)
         
        create_directories([self.Config.artifacts_root])

    def get_prepare_base_model_config(self) ->PrepareBaseModelConfig:
        config = self.Config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir= config.root_dir,
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.Params.IMAGE_SIZE,
            params_learning_rate = float(self.Params.LEARNING_RATE),
            params_include_top = self.Params.INCLUDE_TOP,
            params_weights = self.Params.WEIGHTS,
            params_classes = self.Params.CLASSES
              )
        return prepare_base_model_config



In [10]:
def save_model(self, path, model):
    model.save(path)

In [30]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from pathlib import Path

In [31]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config
    

    def get_base_model(self):
        self.model = tf.keras.applications.resnet50.ResNet50(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top

        )

        self.save_model(path=self.config.base_model_path,model = self.model)



    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
        prediction = tf.keras.layers.Dense(classes, activation="softmax")(x)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all= True,
            freeze_till=None,
            learning_rate= self.config.params_learning_rate

        )
        self.save_model(path = self.config.updated_base_model_path,model = self.full_model)
    
    @staticmethod
    def save_model(path, model: tf.keras.Model):
        model.save(path)


In [32]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
     

except Exception as e:
    raise e   

[2026-01-17 01:07:28,814: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-17 01:07:28,846: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-17 01:07:28,852: INFO: common: created directory at: artifacts]
[2026-01-17 01:07:28,852: INFO: common: created directory at: artifacts/prepare_base_model]
[2026-01-17 01:07:32,891: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                              